In [1]:
from ddpg_agent import Agent
from environment import Environment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import torch

In [2]:
critic_checkpoint_path = "checkpoints/crawler_checkpoint_critic.pth"  
actor_checkpoint_path = "checkpoints/crawler_checkpoint_actor.pth"  
env = Environment(env_path = "Crawler_Linux/Crawler.x86_64")
agent = Agent(num_agents = env.num_agents, state_size=env.state_size, action_size=env.action_size, random_seed=2, is_crawler = True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: CrawlerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 129
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 20
        Vector Action descriptions: , , , , , , , , , , , , , , , , , , , 


Reacher environment Initialized
(12, 129)
Number of agents: 12
Size of each action: 20
There are 12 agents. Each observes a state with length: 129
Creating 12 agents. Each observes a state with length: 129 and has an action space of size: 20
Creating Crawler Model
Creating Crawler Model


In [3]:
#Train the agent with DDPG
def ddpg(n_episodes=1000, max_t=1000, print_every=100, target_score = 34.0):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        states = env.reset()
        agent.reset()
        score = 0
        for t in range(max_t):
            actions = agent.act(states)
            next_states, rewards, dones, _ = env.step(actions)
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            score += np.average(rewards)
            if any(dones):
                #print("break after {} time steps".format(t))
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=target_score:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), actor_checkpoint_path)
            torch.save(agent.critic_local.state_dict(), critic_checkpoint_path)
            np.savez("scores_crawler.npz",np.array(scores))
            break
     # Save Agent at the end of training
    torch.save(agent.actor_local.state_dict(), actor_checkpoint_path)
    torch.save(agent.critic_local.state_dict(), critic_checkpoint_path)
    return scores

scores = ddpg()
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/home/jdgalviss/repos/robotic_arm_control_RL/ddpg_agent.py:126: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1) # Gradient Clipping


Episode 9	Average Score: 2.30

KeyboardInterrupt: 